# Model

Our model begins with a Regression Classifier.

In [31]:
conda install xgboost

Note: you may need to restart the kernel to use updated packages.

Solving environment: ...working... done

## Package Plan ##

  environment location: c:\Users\valeh\anaconda3

  added / updated specs:
    - xgboost


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _py-xgboost-mutex-2.0      |            cpu_0          11 KB  conda-forge
    libxgboost-2.0.3           |   cpu_h0f92c63_0         1.3 MB  conda-forge
    py-xgboost-2.0.3           |cpu_py311h997e2c4_0         357 KB  conda-forge
    xgboost-2.0.3              |cpu_py311h997e2c4_0          16 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         1.7 MB

The following NEW packages will be INSTALLED:

  _py-xgboost-mutex  conda-forge/win-64::_py-xgboost-mutex-2.0-cpu_0 
  libxgboost         conda-forge/win-64::libxgboost-2.0.3-cpu_h0f92c63_0 

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): conda.anaconda.org:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): conda.anaconda.org:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): repo.anaconda.com:443
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pkgs/r/win-64/current_repodata.json HTTP/1.1" 304 0
DEBUG:urllib3.connectionpool:https://conda.anaconda.org:443 "GET /conda-forge/noarch/current_repodata.json HTTP/1.1" 200 None
DEBUG:urllib3.connectionpool:https://repo.anaconda.com:443 "GET /pk

# Description

We attempt three models:

1. RandomForestRegressor  
2. LinearRegression  
3. XGBoost

In [3]:
import kagglehub
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import seaborn as sns

# Regression Classifier.
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import pandas as pd

# Statsmodels.
import statsmodels.api as sm
import pandas as pd

In [ ]:
# Load dataset.
# Download latest version
path = kagglehub.dataset_download("lashagoch/life-expectancy-who-updated")
print("Path to dataset files:", path)

file = "Life-Expectancy-Data-Updated.csv"

# TODO:  Correct features and assign target.
data = pd.read_csv(path + "/" + file)
data.columns
X = data[['Infant_deaths', 'Under_five_deaths',
       'Adult_mortality', 'Alcohol_consumption', 'Hepatitis_B', 'Measles',
       'BMI', 'Polio', 'Diphtheria', 'Incidents_HIV', 'GDP_per_capita',
       'Population_mln', 'Thinness_ten_nineteen_years',
       'Thinness_five_nine_years', 'Schooling', 'Economy_status_Developed',
       'Economy_status_Developing']]

y = data['Life_expectancy']

# Split data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## Method 1

RandomForestRegressor

In [4]:
# Load dataset.
# Download latest version
path = kagglehub.dataset_download("lashagoch/life-expectancy-who-updated")
print("Path to dataset files:", path)

file = "Life-Expectancy-Data-Updated.csv"

# TODO:  Correct features and assign target.
data = pd.read_csv(path + "/" + file)
data.columns
X = data[['Infant_deaths', 'Under_five_deaths',
       'Adult_mortality', 'Alcohol_consumption', 'Hepatitis_B', 'Measles',
       'BMI', 'Polio', 'Diphtheria', 'Incidents_HIV', 'GDP_per_capita',
       'Population_mln', 'Thinness_ten_nineteen_years',
       'Thinness_five_nine_years', 'Schooling', 'Economy_status_Developed',
       'Economy_status_Developing']]

y = data['Life_expectancy']

# Split data into training and testing sets.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create a Logistic Regression classifier instance.
model = RandomForestRegressor(oob_score=True)

# Train the model.
model.fit(X_train, y_train)

# Make predictions.
y_pred = model.predict(X_test)

# Evaluate the model.
print(model.oob_score_)
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Accuracy:  {accuracy}')

100%|██████████| 104k/104k [00:00<00:00, 1.45MB/s]

Extracting files...
Path to dataset files: C:\Users\valeh\.cache\kagglehub\datasets\lashagoch\life-expectancy-who-updated\versions\1


0.996240193583538


## Method 2

Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# Prepare data.
X = np.array([1, 2, 3, 4, 5]).reshape(-1, 1)    # Single feature.
y = np.array([2, 4, 5, 4, 5])

# Create a Linear Regression model object.
model = LinearRegression()

# Fit the model to data.
model.fit(X, y)

# Make predictions.
new_X = np.array([6, 7]).reshape(-1, 1)
predictions = model.predict(new_X)

# Access model attributes.
slope = model.coef_
intercept = model.intercept_

# Print results.
print(f'Coefficient:  {slope}')
print(f'Intercept:  {intercept}')
print(f'Predictions:  {predictions}')

## Model 3

XGBoost

In [ ]:
from xgboost import XGBRegressor

In [ ]:
# Construct XGBoost regression model.
model = XGBRegressor(
    n_estimators=100,       # Number of boosted trees.
    learning_rate=0.1,      # Shrinkage (eta).
    max_depth=6,            # Tree depth.
    subsample=1.0,          # Row sampling.
    colsample_bytree=1.0,   # Feature sampling.
    random_state=0
)

# Fit model and predict results.
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

## Model 4 (Optional)

StatsModel

In [ ]:
# # Load dataset.
# # TODO:  Correct features and assign target.
# data = pd.DataFrame(data)
# X = data[['feature1', 'feature2']]
# y = data['target']

# # Add a constant (intercept) to the features.
# X = sm.add_constant(X)

# # Create and fit a Logistic Regression model.
# model = sm.Logit(y, X)
# results = model.fit()

# # Print the summary of results.
# print(results.summary())

# # Make predictions (requires a threshold for classification).
# predictions = results.predict(X)
# binary_predictions = (predictions > 0.5).astype(int)
# print(binary_predictions)